# Host-guest usage with stk

by: Andrew Tarzia

### Update 25/02/24

Installation of py3Dmol using `pip install py3Dmol` is required.

#### Using latest version of `SpinDry` and `stk`.

- Changes in `SpinDry.Molecule` means we now use `spd.Molecule()` not `spd.Molecule.init()`
- `spd.Spinner.get_conformers` should now take `spd.Supramolecule`, which we initialise from components.
- To then access those components, you must do that through the `Supramolecule` interface.
- Finally, we had to change to using the `stk.complex.Guest` interface.
- See https://stk.readthedocs.io/en/stable/_autosummary/stk.host_guest.Complex.html
- See https://github.com/andrewtarzia/SpinDry/blob/main/examples/

# Imports 

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

from rdkit import Chem 
from rdkit.Chem import AllChem as rdkit
from collections import defaultdict
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDistGeom
IPythonConsole.ipython_3d = True

import py3Dmol
from IPython.display import Image
import matplotlib.pyplot as plt
import subprocess
import time
import stk
import stko
import spindry as spd
%matplotlib inline

# Some useful functions 

In [ ]:
def show_stk_mol(stk_mol):
    data = rdkit.MolToMolBlock(stk_mol.to_rdkit_mol())
    p = py3Dmol.view(
        data=data,
        style={'stick':{'colorscheme':'cyanCarbon'}}, 
        width=400,
        height=400,
    )
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    p.show()

Working:

In [ ]:
# Produce a Pd+2 atom with 4 functional groups.
palladium_atom = stk.BuildingBlock(
    smiles='[Pd+2]',
    functional_groups=(
        stk.SingleAtom(stk.Pd(0, charge=2))
        for i in range(4)
    ),
    position_matrix=[[0., 0., 0.]],
)

# Build a building block with two functional groups using
# the SmartsFunctionalGroupFactory.
bb1 = stk.BuildingBlock(
    smiles=(
        'C1=NC=CC(C2=CC=CC(C3=CC=NC=C3)=C2)=C1'
    ),
    functional_groups=[
        stk.SmartsFunctionalGroupFactory(
            smarts='[#6]~[#7X2]~[#6]',
            bonders=(1, ),
            deleters=(),
        ),
    ],
)

cage1 = stk.ConstructedMolecule(
    stk.cage.M6L12Cube(
        building_blocks=(palladium_atom, bb1),
        # Ensure that bonds between the GenericFunctionalGroups
        # of the ligand and the SingleAtom functional groups
        # of the metal are dative.
        reaction_factory=stk.DativeReactionFactory(
            stk.GenericReactionFactory(
                bond_orders={
                    frozenset({
                        stk.GenericFunctionalGroup,
                        stk.SingleAtom
                    }): 9
                }
            )
        ),
        optimizer=stk.MCHammer(num_steps=2000),
    )
)

In [ ]:
show_stk_mol(cage1)

In [ ]:
stk_guest = stk.BuildingBlock('C1C(O[H])C2C(C=CC(=C2)CP([H])[H])CC1')
show_stk_mol(stk_guest)

In [ ]:
cage_atoms = [
    (atom.get_id(), atom.__class__.__name__)
    for atom in cage1.get_atoms()
]
stk_mol_atoms = [
    (atom.get_id(), atom.__class__.__name__)
    for atom in stk_guest.get_atoms()
]

In [ ]:
host = spd.Molecule(
    atoms=(
        spd.Atom(id=i[0], element_string=i[1])
        for i in cage_atoms
    ),
    bonds=(),
    position_matrix=cage1.get_position_matrix(),
)
guest = spd.Molecule(
    atoms=(
        spd.Atom(id=i[0], element_string=i[1])
        for i in stk_mol_atoms
    ),
    bonds=(),
    position_matrix=stk_guest.get_position_matrix(),
)

In [ ]:
print(host, guest)

In [ ]:
cg = spd.Spinner(
    step_size=0.5,
    rotation_step_size=5,
    num_conformers=100,
    max_attempts=2000,
)

In [ ]:
supramolecule = spd.SupraMolecule.init_from_components(components=(host, guest))

In [ ]:
energies = {}
for conformer in cg.get_conformers(supramolecule):
    print(conformer)
    print(conformer.get_cid(), conformer.get_potential())
    for i, comp in enumerate(list(conformer.get_components())):
        if i == 0:
            cage1 = cage1.with_position_matrix(comp.get_position_matrix())
        elif i == 1:
            stk_guest = stk_guest.with_position_matrix(comp.get_position_matrix())

    complex_mol = stk.ConstructedMolecule(
        topology_graph=stk.host_guest.Complex(
            host=stk.BuildingBlock.init_from_molecule(cage1), 
            guests=stk.host_guest.Guest(stk_guest),
        )
    )
    complex_mol.write(
        f'hg_example_output/hg_conf_{conformer.get_cid()}.mol'
    )
    energies[conformer.get_cid()] = stko.UFFEnergy(ignore_inter_interactions=False).get_energy(complex_mol)

In [ ]:
energies